In [147]:
pip install --upgrade optbinning

In [222]:
# Importamos las librerias a utilizar
import pandas as pd
from sklearn.model_selection import train_test_split
from optbinning import OptimalBinning
# acceder a drive
from google.colab import drive
drive.mount("/content/drive/")
# carga de  base de datos original
data = pd.read_csv("/Base PFAE para automatizacion IV (1) (1).csv")

# Filtrar los datos en "producto_fin" igual a "Nuevo"
# aqui voy filtrar las distintas poblaciones, p. ej. con covid sin covid
filtered_data = data[(data['producto_fin'] == 'Nuevo') & (data['base'] == 'formalizado')]
# Variables independientes (X) y variable objetivo (y)
X = filtered_data.drop(columns=["target_90_12m_fin2"])
y = filtered_data["target_90_12m_fin2"]

# Dividir los datos en conjuntos de entrenamiento y validación,
#divido el train para empezar a ver que variables son predictivas en esta poblacion
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# agrego la variable entreamiento para un cruce posterior
X_train = X_train.assign(entrenamiento =1)
X_entreamiento=X_train[["folio","entrenamiento"]]
# Usar la columna "folio" como clave de unión para realizar el cruce
# cruzo mi base original con el train para marcar cuales créditos entran en mi muestra de entramiento
merged_data2 = pd.merge(filtered_data, X_entreamiento, on='folio', how='left')
# filtramos la base de entramiento para hacer los WOE
base_woe=merged_data2[merged_data2['entrenamiento'] == 1]

# Importar las bibliotecas necesarias nuevamente
# estas librerias nos ayudan a hacer los rangos de manera automática y
# para calcular el IV de manera fácil
from optbinning import OptimalBinning
# Lista de columnas para convertir a numéricas y manejar errores
columns_to_convert = ['PF_SALDO_ACTUAL', 'PF_CREDITO_MAXIMO', 'PF_CREDITOS_TOTALES', 'PF_CREDITOS_ABIERTOS', 'PF_MAX_MOP_U6M', 'PF_MAX_MOP_U9M', 'PF_MAX_MOP_U12M', 'PF_UTIL_TDREV', 'PF_CANT_MES_UATR_M60D', 'PF_MAX_EDAD_PRODUC', 'pf_pct_lc_abierto_u3m', 'pf_pct_lc_abierto_u6m', 'pf_pct_lc_BANCO', 'pf_pct_lc_OTROS', 'pf_pct_lc_FINANCIERAS', 'pf_COCIENTE_U3M_50K', 'pf_COCIENTE_U3M_250K', 'PF_MOP_PONDERADO', 'PF_NUM_INQ_3M_ALL_NOTU', 'PF_NUM_INQ_6M_ALL_NOTU', 'PF_NUM_INQ_9M_ALL_NOTU', 'PF_NUM_INQ_12M_ALL_NOTU']
# Convertir y manejar errores para las columnas especificadas
for column in columns_to_convert:
    base_woe[column] = pd.to_numeric(base_woe[column], errors='coerce')
    # Imprime las columnas que estan en merged
for column in base_woe.columns:
    print(column)
# Define las variables que deseas incluir en X para el calculo de los bins
variables = ['PF_SALDO_ACTUAL',
'PF_CREDITO_MAXIMO',
'PF_CREDITOS_TOTALES',
'PF_CREDITOS_ABIERTOS',
'PF_MAX_MOP_U6M',
'PF_MAX_MOP_U9M',
'PF_MAX_MOP_U12M',
'PF_UTIL_TDREV',
'PF_CANT_MES_UATR_M60D',
'PF_MAX_EDAD_PRODUC',
'pf_pct_lc_abierto_u3m',
'pf_pct_lc_abierto_u6m',
'pf_pct_lc_BANCO',
'pf_pct_lc_OTROS',
'pf_pct_lc_FINANCIERAS',
'pf_COCIENTE_U3M_50K',
'pf_COCIENTE_U3M_250K',
'PF_MOP_PONDERADO',
'PF_NUM_INQ_3M_ALL_NOTU',
'PF_NUM_INQ_6M_ALL_NOTU',
'PF_NUM_INQ_9M_ALL_NOTU',
'PF_NUM_INQ_12M_ALL_NOTU']


#Inicializa el objeto OptimalBinning para cada variable
#aqui empiezo a ver los rangos y el IV, IV mayores a 0.10 los considero para la regresión
optimal_binnings = {}
for variable in variables:
    optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")
    optb.fit(base_woe[variable], base_woe['target_90_12m_fin2'])
    optimal_binnings[variable] = optb

# Obtiene los resultados del binning óptimo para cada variable
binning_tables = {}
for variable, optb in optimal_binnings.items():
    binning_table = optb.binning_table.build()
    binning_tables[variable] = binning_table

# Visualización de las tablas de binning óptimo para cada variable
for variable, binning_table in binning_tables.items():
    print(f"Binning Table para {variable}:\n")
    print(binning_table)


#aqui sustiyo la variable cruda por los woe, uso los rangos que me da python como referencia
# y los retrabajo en excel como confirmacion , hago la regresion con los woe
woe_consultas = []
for row in base_woe['PF_NUM_INQ_6M_ALL_NOTU']:
  if row < 0 : woe_consultas.append(0.218895620616713)
  elif row < 2.0: woe_consultas.append(1.86957649158486)
  elif row < 6.0: woe_consultas.append(0.416142827627343)
  elif row < 12.0: woe_consultas.append(0.0514997140394342)
  elif row < 20.0: woe_consultas.append(-0.422958265555682)
  elif row < 5000000.0: woe_consultas.append(-0.62675885402676)
  else: woe_consultas.append(0.218895620616713)


base_woe["woe_consultas"] = woe_consultas

woe_utl = []
for row in base_woe['PF_UTIL_TDREV']:
  if row < 0.01 : woe_utl.append(-0.0479523649322257)
  elif row < 0.2: woe_utl.append(0.938018287579919)
  elif row < 0.48: woe_utl.append(0.744646894599379)
  elif row < 0.88: woe_utl.append(-0.122853673105343)
  else: woe_utl.append(-0.710640338007462)

base_woe["woe_utl"] = woe_utl

woe_edad = []
for row in base_woe['PF_MAX_EDAD_PRODUC']:
  if row < 0 : woe_edad.append(0.0114314326778391)
  elif row < 17: woe_edad.append(0.0224253881832208)
  elif row < 36: woe_edad.append(0.0106331074791457)
  elif row < 48: woe_edad.append(0.00249054426334374)
  elif row < 100: woe_edad.append(0.0000146997756979677)
  elif row < 200: woe_edad.append(0.018330002633592)
  else: woe_edad.append(0.0316294870283295)

base_woe["woe_edad"] = woe_edad

import statsmodels.api as sm

# aqui hago la regresion, uso statmodels
x_tr = base_woe[['woe_consultas', 'woe_utl','woe_edad']]
y_tr = base_woe[['target_90_12m_fin2']]

# construyo el modelo
log_reg = sm.Logit(y_tr, x_tr).fit()

# imprimo los resultados del modelo
print(log_reg.summary())

# veo los resultados, analizo la significancia de cada variable


# aqui empiezo a trabajar con la base de validacion
base_test=merged_data2[merged_data2['entrenamiento'] != 1]
# Convertir y manejar errores para las columnas especificadas
for column in columns_to_convert:
    base_test[column] = pd.to_numeric(base_test[column], errors='coerce')


# aqui sustituyo los woe en la base de validacion
woe_consultas = []
for row in base_test['PF_NUM_INQ_6M_ALL_NOTU']:
  if row < 0 : woe_consultas.append(0.218895620616713)
  elif row < 2.0: woe_consultas.append(1.86957649158486)
  elif row < 6.0: woe_consultas.append(0.416142827627343)
  elif row < 12.0: woe_consultas.append(0.0514997140394342)
  elif row < 20.0: woe_consultas.append(-0.422958265555682)
  elif row < 5000000.0: woe_consultas.append(-0.62675885402676)
  else: woe_consultas.append(0.218895620616713)

base_test["woe_consultas"] = woe_consultas

woe_utl = []
for row in base_test['PF_UTIL_TDREV']:
  if row < 0.01 : woe_utl.append(-0.0479523649322257)
  elif row < 0.2: woe_utl.append(0.938018287579919)
  elif row < 0.48: woe_utl.append(0.744646894599379)
  elif row < 0.88: woe_utl.append(-0.122853673105343)
  else: woe_utl.append(-0.710640338007462)

base_test["woe_utl"] = woe_utl


woe_edad = []
for row in base_test['PF_MAX_EDAD_PRODUC']:
  if row < 0 : woe_edad.append(0.0114314326778391)
  elif row < 17: woe_edad.append(0.0224253881832208)
  elif row < 36: woe_edad.append(0.0106331074791457)
  elif row < 48: woe_edad.append(0.00249054426334374)
  elif row < 100: woe_edad.append(0.0000146997756979677)
  elif row < 200: woe_edad.append(0.018330002633592)
  else: woe_edad.append(0.0316294870283295)

base_test["woe_edad"] = woe_edad

# loading the testing dataset

# aqui empiezo con la validacion del modelo
base_test

# defining the dependent and independent variables
Xtest = base_test[['woe_utl', 'woe_consultas','woe_edad']]
ytest = base_test['target_90_12m_fin2']

# performing predictions on the test dataset
yhat = log_reg.predict(Xtest)
prediction = list(map(round, yhat))

# comparing original and predicted values of y
print('Actual values', list(ytest.values))
print('Predictions :', prediction)


from sklearn.metrics import (confusion_matrix,
                           accuracy_score)

# confusion matrix
cm = confusion_matrix(ytest, prediction)
print ("Confusion Matrix : \n", cm)

# accuracy score of the model
print('Test accuracy = ', accuracy_score(ytest, prediction))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
base
folio
fecha_contrato
vintage_num
monto_operado
monto_neto
producto_fin
plazo
company_type
tipo_cred
folio_n
fecha_n
producto_n
clasificacion_n
target_90_12m_fin2
no_control_bc_pf
no_control_bc_pm
FROM
from_tipo
pf_hit_tu
PF_SALDO_ACTUAL
PF_CREDITO_MAXIMO
PF_CREDITOS_TOTALES
PF_CREDITOS_ABIERTOS
PF_MAX_MOP_U6M
PF_MAX_MOP_U9M
PF_MAX_MOP_U12M
SUM_SA
SUM_LC
PF_UTIL_TDREV
PF_CANT_MES_UATR_M60D
PF_MAX_EDAD_PRODUC
S_LC_ABIERTO
S_LC_ABIERTO_U3M
S_LC_ABIERTO_U6M
S_LC_BANCO
S_LC_FINANCIERAS
S_LC_OTROS
pf_pct_lc_abierto_u3m
pf_pct_lc_abierto_u6m
pf_pct_lc_BANCO
pf_pct_lc_OTROS
pf_pct_lc_FINANCIERAS
pf_COCIENTE_U3M_50K
pf_COCIENTE_U3M_250K
COCIENTE_U3M_500K
S_LC_U18M
S_lc_ponderado_mop18
PF_MOP_PONDERADO
PF_NUM_INQ_3M_ALL_NOTU
PF_NUM_INQ_6M_ALL_NOTU
PF_NUM_INQ_9M_ALL_NOTU
PF_NUM_INQ_12M_ALL_NOTU
entrenamiento


<ipython-input-222-6f26809f5725>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_woe[column] = pd.to_numeric(base_woe[column], errors='coerce')


Binning Table para PF_SALDO_ACTUAL:

                            Bin  Count  Count (%)  Non-event  Event  \
0                 (-inf, -0.50)    106   0.236607         85     21   
1             [-0.50, 82259.50)     41   0.091518         37      4   
2         [82259.50, 285121.00)     68   0.151786         51     17   
3       [285121.00, 3459067.00)    182   0.406250        136     46   
4             [3459067.00, inf)     51   0.113839         37     14   
5                       Special      0   0.000000          0      0   
6                       Missing      0   0.000000          0      0   
Totals                             448   1.000000        346    102   

        Event rate       WoE        IV        JS  
0         0.198113  0.176663  0.007028  0.000877  
1         0.097561  1.003158  0.067935  0.008153  
2         0.250000 -0.122854  0.002367  0.000296  
3         0.252747 -0.137452  0.007961  0.000994  
4         0.274510 -0.249605  0.007568  0.000944  
5         0.00000

<ipython-input-222-6f26809f5725>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_woe["woe_consultas"] = woe_consultas
<ipython-input-222-6f26809f5725>:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_woe["woe_utl"] = woe_utl
<ipython-input-222-6f26809f5725>:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

# Nueva sección